<a href="https://colab.research.google.com/github/SooinJung/SLM_project_redits/blob/main/ko_gemma_2_9b_it.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install the libraries
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Required when training models/data that are gated on HuggingFace, and required for pushing models to HuggingFace
from huggingface_hub import notebook_login

notebook_login()

## 모델 불러오기
4비트 양자화 or 8비트 양자화 선택 가능

In [4]:
# pip install bitsandbytes
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch


model_id = "rtzr/ko-gemma-2-9b-it"
#quantization_config_8bit = BitsAndBytesConfig(load_in_8bit=True)
quantization_config_4bit = BitsAndBytesConfig(load_in_4bit=True)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    # quantization_config=quantization_config_8bit,
    quantization_config=quantization_config_4bit,
    low_cpu_mem_usage=True, # cpu 메모리 적게 사용하기
)


config.json:   0%|          | 0.00/852 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/39.1k [00:00<?, ?B/s]

model-00001-of-00010.safetensors:   0%|          | 0.00/1.92G [00:00<?, ?B/s]

model-00002-of-00010.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00003-of-00010.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

model-00008-of-00010.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00009-of-00010.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00010-of-00010.safetensors:   0%|          | 0.00/705M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [5]:
model.eval()
instruction = "서울의 유명한 관광 코스를 만들어줄래?"

messages = [
    {"role": "user", "content": f"{instruction}"}
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<end_of_turn>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=2048,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

print(tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True))

## 서울 탐험 코스: 3일 완벽 가이드

**1일차: 역사와 문화 속으로**

* **오전:** 경복궁 & 국립고궁박물관: 조선시대의 웅장함을 느껴보세요. 궁궐의 아름다운 건축물과 역사를 감상하고, 국립고궁박물관에서는 조선시대 왕실의 생활과 문화를 더 깊이 이해할 수 있습니다.
* **점심:** 북촌 한옥마을: 전통적인 한옥 건축물 사이를 거닐며, 맛있는 한식을 즐기세요. 북촌의 아름다운 풍경과 전통 분위기를 만끽하며 점심 식사를 즐기세요.
* **오후:** 인사동 & 북촌: 다양한 공예품과 전통 예술품을 둘러보세요. 인사동 거리의 작은 골목길을 따라 걷고, 북촌 한옥마을에서 전통 공예품과 예술품을 구경하며 쇼핑을 즐겨보세요.
* **저녁:** 광장시장: 서울의 다양한 맛을 경험하세요. 광장시장의 다채로운 음식과 쇼핑 환경을 즐기며 저녁 식사를 하세요.

**2일차: 현대의 pulse를 느껴보세요**

* **오전:** 명동: 패션과 쇼핑의 중심지 명동에서 즐거운 시간을 보내세요. 다양한 브랜드와 쇼핑몰을 방문하며 명동의 활기 넘치는 분위기를 만끽하세요.
* **점심:** 쌈밥 & 떡볶이: 현대적인 분위기 속에서 맛있는 점심 식사를 즐기세요. 명동의 다양한 음식점에서 쌈밥이나 떡볶이를 맛보세요.
* **오후:**  Gangnam: 서울의 화려함을 경험하세요. 쇼핑몰과 고급 레스토랑이 모여있는 강남을 방문하고, 젊은 감성을 느껴보세요. 
* **저녁:** 롯데월드타워: 서울의 야경을 감상하세요. 롯데월드타워에서 서울의 아름다운 야경을 감상하며 저녁 식사를 즐기세요.

**3일차: 자연 속에서 휴식**

* **오전:** 북한산: 서울 근교의 아름다운 자연을 즐겨보세요. 북한산에서 등산을 즐기고, 자연 속에서 휴식을 취하세요.
* **점심:** 북한산 근처 레스토랑: 자연을 감상하며 맛있는 점심 식사를 즐기세요. 북한산 근처의 레스토랑에서 맛있는 점심 식사를 즐기며 자연을 만끽하세요.
* **오후:** 서울숲: 도시 속 녹음 공간을 탐험하세요. 서울숲에서 산책을